### Description

This notebook is the validation of the `grad2` and `counting number` accumulated in the ad graph. 
\
We verify these terms by comparing those we obtian from rendering the entire image and those we obtain from rendering with only one single ray. For simplicity we use direct illumination.

In [1]:
import mitsuba as mi
import drjit as dr
import matplotlib.pyplot as plt
import numpy as np


mi.set_variant('llvm_ad_rgb')

scene = mi.load_file('scenes/cbox_ray.xml')



Camera settings

In [2]:
ambient_range = 0.75
ambient_ray_count = 256

# Camera origin in world space
cam_origin = mi.Point3f(0, 1, 3)

# Camera view direction in world space
cam_dir = dr.normalize(mi.Vector3f(0, -0.5, -1))

# Camera width and height in world space
cam_width  = 2.0
cam_height = 2.0


Ray set generation given image resolution

In [3]:
def get_ray_origin(image_res_):
    # Construct a grid of 2D coordinates
    x, y = dr.meshgrid(
    dr.linspace(mi.Float, -cam_width  / 2,   cam_width / 2, image_res_[0]),
    dr.linspace(mi.Float, -cam_height / 2,  cam_height / 2, image_res_[1])
    )

    # Ray origin in local coordinates
    ray_origin_local = mi.Vector3f(x, y, 0)

    # Ray origin in world coordinates
    ray_origin_ = mi.Frame3f(cam_dir).to_world(ray_origin_local) + cam_origin
    return ray_origin_



Ambient occlusion

In [4]:
def run(si):
    # Loop iteration counter
    i = mi.UInt32(0)

    # Accumulated result
    result = mi.Float(0)

    # Initialize the loop state (listing all variables that are modified inside the loop)
    loop = mi.Loop(name="", state=lambda: (rng, i, result))

    while loop(si.is_valid() & (i < ambient_ray_count)):
        # 1. Draw some random numbers
        sample_1, sample_2 = rng.next_float32(), rng.next_float32()
        
        # 2. Compute directions on the hemisphere using the random numbers
        wo_local = mi.warp.square_to_uniform_hemisphere([sample_1, sample_2])

        # Alternatively, we could also sample a cosine-weighted hemisphere
        # wo_local = mi.warp.square_to_cosine_hemisphere([sample_1, sample_2])
        
        # 3. Transform the sampled directions to world space
        wo_world = si.sh_frame.to_world(wo_local)

        # 4. Spawn a new ray starting at the surface interactions
        ray_2 = si.spawn_ray(wo_world)
        
        # 5. Set a maximum intersection distance to only account for the close-by geometry
        ray_2.maxt = ambient_range

        # 6. Accumulate a value of 1 if not occluded (0 otherwise)
        result[~scene.ray_test(ray_2)] += 1.0
        
        # 7. Increase loop iteration counter
        i += 1

    # Divide the result by the number of samples
    result = result / ambient_ray_count
    return result


Direct illumination

In [5]:
def mis_weight(pdf_a, pdf_b):
    """
    Compute the Multiple Importance Sampling (MIS) weight given the densities
    of two sampling strategies according to the power heuristic.
    """
    a2 = dr.sqr(pdf_a)
    b2 = dr.sqr(pdf_b)
    w = a2 / (a2 + b2)
    return dr.detach(dr.select(dr.isfinite(w), w, 0))

def direct(si,ray):
    # Initialize the random number generator
    rng = mi.PCG32(size=1)

    # Loop iteration counter
    i = mi.UInt32(0)

    # Accumulated result
    result = mi.Color3f(0.0)
    max_depth = 1

    active = si.is_valid()

    # Initialize the loop state (listing all variables that are modified inside the loop)
    # loop = mi.Loop(name="", state=lambda: (rng, i, result, ray, active))

    # while loop(active & (i < max_depth)):
    ray_reparam = mi.Ray3f(dr.detach(ray))
    bsdf_ctx = mi.BSDFContext()

    # ---------------------- Direct emission ----------------------
    si = scene.ray_intersect(ray_reparam, active)
    result += si.emitter(scene).eval(si)

    # ------------------ Emitter sampling -------------------

    # Should we continue tracing to reach one more vertex?
    active_next = si.is_valid()
    # Get the BSDF. Potentially computes texture-space differentials.
    bsdf = si.bsdf(ray)

    # Detached emitter sample
    active_em = active_next & mi.has_flag(
        bsdf.flags(), mi.BSDFFlags.Smooth)
    sample_2d = mi.Point2f(rng.next_float32(), rng.next_float32())

    ds, weight_em = scene.sample_emitter_direction(
            si, sample_2d, True, active_em)
    active_em &= dr.neq(ds.pdf, 0.0)
    
    # Reparameterize the ray
    ray_em_det = 1.0
    
    # Compute MIS
    wo = si.to_local(ds.d)
    bsdf_value_em, bsdf_pdf_em = bsdf.eval_pdf(bsdf_ctx, si, wo, active_em)
    mis_em = dr.select(ds.delta, 1.0, mis_weight(ds.pdf, bsdf_pdf_em))

    result += bsdf_value_em * weight_em * ray_em_det * mis_em

    # ------------------ BSDF sampling -------------------

    # Detached BSDF sample

    sample_2d = mi.Point2f(rng.next_float32(active_next), rng.next_float32(active_next))
    bsdf_sample, bsdf_weight = bsdf.sample(bsdf_ctx, si, rng.next_float32(active_next),
                                            sample_2d, active_next)
    ray_bsdf = si.spawn_ray(si.to_world(bsdf_sample.wo))
    active_bsdf = active_next & dr.any(dr.neq(bsdf_weight, 0.0))

    # Reparameterize the ray
    ray_bsdf_det = 1.0
    
    # Illumination
    si_bsdf = scene.ray_intersect(ray_bsdf, active_bsdf)
    L_bsdf = si_bsdf.emitter(scene).eval(si_bsdf, active_bsdf)

    # Compute MIS
    ds = mi.DirectionSample3f(scene, si_bsdf, si)
    delta = mi.has_flag(bsdf_sample.sampled_type, mi.BSDFFlags.Delta)
    emitter_pdf = scene.pdf_emitter_direction(
        si, ds, active_bsdf & ~delta)
    mis_bsdf = mis_weight(bsdf_sample.pdf, emitter_pdf)

    result += L_bsdf * bsdf_weight * ray_bsdf_det * mis_bsdf
    # print(dr.width(result))
    
    # result += L_bsdf * bsdf_weight  # MASKED FOR ACCEPTED STATES

    i+=1

    # # Divide the result by the number of samples
    # result = result / ambient_ray_count
    return result


In [6]:
def mse(img, ref):
    return dr.mean(dr.sqr(img - ref))

High res image 
\
256*256 , just to show the clear testing scene

In [7]:
image_res = [256,256]
ray_origin_h= get_ray_origin(image_res)
ray_h = mi.Ray3f(o=ray_origin_h, d=cam_dir)
si_h = scene.ray_intersect(ray_h)

img_h = direct(si_h,ray_h)
img_h_np = np.array(img_h).reshape(image_res[0], image_res[1], 3)
img_h_np = mi.TensorXf(img_h_np)
mi.util.convert_to_bitmap(img_h_np)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 192 KiB of image data ]
]

Validation is done on low res 10 * 10

In [8]:
image_res = [10,10]
ray_origin= get_ray_origin(image_res)



##### 1. Initial image
\
Res: 10 * 10, with the entire ray set

In [9]:
ray_set = mi.Ray3f(o=ray_origin, d=cam_dir)
si_set = scene.ray_intersect(ray_set)

img_ref = direct(si_set,ray_set)
img_ref_np = np.array(img_ref).reshape(image_res[0], image_res[1], 3)
img_ref_np = mi.TensorXf(img_ref_np)
mi.util.convert_to_bitmap(img_ref_np)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [10, 10],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 300 B of image data ]
]

##### 2. Change the color of green to blue

In [10]:
params = mi.traverse(scene)
# key = 'green.reflectance.value'
key = 'color_checkerboard.color0.value'

# Save the original value
param_ref = mi.Color3f(params[key])

# Set another color value and update the scene
params[key] = mi.Color3f(0.01, 0.2, 0.9)
dr.enable_grad(params[key])
params.update();

##### 3. The changed image
Here the image is rendered with the entire ray set
\
grad, grad2, counter is also recorded

In [11]:
dr.set_grad(params[key], 0)
si_init = scene.ray_intersect(ray_set)
img_init = direct(si_set,ray_set)
img_init_np = np.array(img_init).reshape(image_res[0], image_res[1], 3)
img_init_np = mi.TensorXf(img_init_np)
mi.util.convert_to_bitmap(img_init_np)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [10, 10],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 300 B of image data ]
]

In [12]:
loss = mse(img_init,img_ref)
# Backpropagate through the rendering process
dr.backward(loss, flags = dr.ADFlag.BackPropVarianceCounter | dr.ADFlag.ClearVertices)
g = dr.grad(params[key])
g2 = dr.grad2(params[key])
c = dr.counter(params[key])
g = np.array(g, dtype=np.float64)
g2 = np.array(g2, dtype=np.float64)
c = np.array(c, dtype=np.float64)
print(g)
print(g2)
print(c)

[[-1.43948507 -0.6731441  -0.01961632]]
[[6.85627639e-01 1.49930492e-01 1.27323656e-04]]
[[7. 7. 7.]]


##### 4. Ray by ray rendering
The image is rendered 1 ray 1 pixel per iteration, and accumulated to form the final rendering. It should be the same as the image rendered with the entire ray set.
\
Here the loss is the mse between the rendered pixel and the corresponding pixel in the init_image.
The grad, grad2, counter, computed for each pixel is recorded.

In [13]:
ray_by_ray = []
grads = []
grad2s = []
counters = []
for i in range(image_res[0]*image_res[1]):
    dr.set_grad(params[key], 0)
    ray2 = mi.Ray3f(o=dr.slice(ray_origin,i), d=cam_dir)
    si2 = scene.ray_intersect(ray2)
    temp = direct(si2,ray2)
    dr.eval(temp)
    loss = mse(temp,dr.slice(img_ref,i))
    dr.backward(loss, flags = dr.ADFlag.BackPropVarianceCounter | dr.ADFlag.ClearVertices)
    g_ = np.array(dr.grad(params[key]))
    g2_ = np.array(dr.grad2(params[key]))
    c_ = np.array(dr.counter(params[key]))
    grads.append(g_)
    grad2s.append(g2_)
    counters.append(c_)
    # print(temp)
    ray_by_ray.append(temp)


In [14]:
img_rbr_np = np.array(ray_by_ray).reshape(image_res[0], image_res[1], 3)
img_rbr_np = mi.TensorXf(img_rbr_np)
mi.util.convert_to_bitmap(img_rbr_np)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [10, 10],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 300 B of image data ]
]

##### 5. Results

a. The image rendered ray by ray should be the same as the one rendered with the entire ray set.
\
b. The sum of grads, grad2s, and counters (recorded in ray by ray process), should be the same as the grad, grad2, and counter we get from the rendering with ray set process.

In [15]:
print(np.array(grads,dtype=np.float64).sum(axis=0))
print(g)
print()

print(np.array(grad2s,dtype=np.float64).sum(axis=0))
print(g2)
print()


print(np.array(counters, dtype=np.float64).sum(axis=0))
print(c)

[[-1.43948518 -0.67314406 -0.01961632]]
[[-1.43948507 -0.6731441  -0.01961632]]

[[6.85627675e-01 1.49930497e-01 1.27323653e-04]]
[[6.85627639e-01 1.49930492e-01 1.27323656e-04]]

[[7. 7. 7.]]
[[7. 7. 7.]]


In [16]:
print(mse(img_rbr_np, img_init_np))

[0.0]
